# Custom acquisition plans

The simplest acqusition plan for a beamline is to move some motor inputs to some positions, and then trigger some detectors. Often, though, we want to investigate behaviors more complex than this. Consider the example of aligning the spectrometer at the Inner-Shell Spectroscopy (ISS) beamline at NSLS-II, which operates by modulating the energy of a beam on a sample and watching the resulting flux rise, peak, and then fall. 

We can build a toy model of this spectrometer using custom acquisition and digestion functions. Let's pretend that we can vary two inputs $\mathbf{x} = (x_1, x_2)$, and that the resolution of the spectrometer is equal to 

$\nu_\sigma = \big (1 + x_1^2 + (x_2 - 1)^2 \big)^{1/2}$

which has a single minimum at $(0,1)$. We can't sample this value directly, rather we can sample how the resulting flux peaks as we vary the energy $\nu$. Let's pretend that it looks like a Gaussian:

$I(\nu) = \exp \Big [ - 0.5 (\nu - \nu_0)^2 / \nu_\sigma^2 \Big ]$

To find the inputs that lead to the tightest spectrum, we need to vary $\mathbf{x}$, scan over $\nu$, and then estimate the resolution for the agent to optimize over. Let's write acquisition and digestion functions to do this: 

In [ ]:
import numpy as np






def acquisition(dofs, inputs, dets):

    uid = yield from bp.list_scan

    for x in inputs:

        res = np.sqrt(1 + np.square(x).sum()) # our resolution is 

        nu_sigma = np.sqrt(1 + x1 ** 2 + (x2 - 1) ** 2)

        flux = np.exp(-0.5*np.square((nu-nu_0)/nu_sigma))



In [ ]:
import numpy as np
from matplotlib import pyplot as plt


nu_0 = 100

nu = np.linspace(90, 110, 256)

for x1, x2 in [(0,0), (-2, 2), (-1,0), (0, 1)]:

    nu_sigma = np.sqrt(1 + x1 ** 2 + (x2 - 1) ** 2)

    flux = np.exp(-0.5*np.square((nu-nu_0)/nu_sigma))

    plt.plot(nu, flux, label=f"(x1, x2) = ({x1}, {x2})")

plt.legend()


To find the inputs that lead to the tightest spectrum, we need to vary $\mathbf{x}$, scan over $\nu$, and then estimate the resolution for the agent to optimize over. Let's write acquisition and digestion functions to do this: 

In [ ]:
import blop 

This determines what the agent will do when it investigates new inputs. 

The only constraint on this function is that it accepts arguments `(dofs, inputs, dets)` as defined previously. 

In [ ]:
%run -i ../../../examples/prepare_bluesky.py
%run -i ../../../examples/prepare_tes_shadow.py

kb_dofs = [kbv.x_rot, kbh.x_rot]
kb_bounds = np.array([[-0.10, +0.10], [-0.10, +0.10]]) 

for dof in kb_dofs:
    dof.kind = "hinted"

In [ ]:

from bloptools.bayesian import Agent
from bloptools.experiments.sirepo import tes

boa = Agent(
       dofs=kb_dofs, 
       bounds=kb_bounds,
       detectors=[w9],
       tasks=[tes.MinBeamWidth, tes.MinBeamHeight, tes.MaxBeamFlux],
       acquisition=tes.acquisition,
       digestion=tes.digestion,
       db=db,
)

RE(boa.initialize(init_scheme='quasi-random', n_init=16))

For each task, we plot the sampled data and the model's posterior with respect to two inputs to the KB mirrors. We can see that each tasks responds very differently to different motors, which is very useful to the optimizer. 

In [ ]:
boa.plot_tasks()

Let's optimize:

In [ ]:
RE(boa.learn(strategy='esti', n_iter=1, n_per_iter=4))
boa.plot_tasks()

The agent has learned that certain dimensions affect different tasks differently!